In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
#from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
#from langchain.embeddings import SentenceTransformerEmbeddings
#from langchain.vectorstores import Chroma 
from langchain_community.vectorstores import Chroma
import os 
from constants import CHROMA_SETTINGS


persist_directory = "db"


def main():
    for root, dirs, files in os.walk("docs"):
        for file in files:
            if file.endswith(".pdf"):
                print(file)
                loader = PDFMinerLoader(os.path.join(root, file))
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    #create embeddings here
    
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    #db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS)
    output_dir = "./db_metadata_v5"
    db = Chroma.from_documents(texts, embeddings, persist_directory=output_dir)
    db.persist()
    db=None

if __name__ == "__main__":
    main()

fastfacts-what-is-climate-change.pdf


In [10]:
import streamlit as st 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64
import textwrap
from langchain_community.embeddings import SentenceTransformerEmbeddings
#from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from constants import CHROMA_SETTINGS
from huggingface_hub import hf_hub_download

#model and tokenizer loading
# checkpoint = "LaMini-T5-738M"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map='auto', torch_dtype=torch.float32)

#model and tokenizer loading
base_model = "LaMini-T5-738M"
weights_location = hf_hub_download(base_model, 'pytorch_model.bin')
config = AutoConfig.from_pretrained(base_model)
with init_empty_weights():
     model = AutoModelForSeq2SeqLM.from_base_model(base_model)
model.tie_weights()
model = load_checkpoint_and_dispatch(model, weights_location, device_map='auto')


# whisper_model = "openai/whisper-tiny"
# weights_location = hf_hub_download(whisper_model, 'pytorch_model.bin')
# config = AutoConfig.from_pretrained(whisper_model)
# with init_empty_weights():
#      model = AutoModelWithLMHead.from_config(config)
# model.tie_weights()
# model = load_checkpoint_and_dispatch(model, weights_location, device_map='auto')

@st.cache_resource
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample=True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

@st.cache_resource
def qa_llm():
    llm = llm_pipeline()
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma(persist_directory="db", embedding_function=embeddings, client_settings=CHROMA_SETTINGS)
    retriever = db.as_retriever()
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
    return qa

def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa(instruction)
    answer = generated_text['result']
    # metadata = generated_text['metadata']
    # for text in generated_text:
        
    #     print(answer)

    # wrapped_text = textwrap.fill(response, 100)
    # return wrapped_text
    return answer,generated_text

def main():
    st.title("Search Your PDF 🐦📄")
    with st.expander("About the App"):
        st.markdown(
            """
            This is a Generative AI powered Question and Answering app that responds to questions about your PDF File.
            """
        )
    question = st.text_area("Enter your Question")
    if st.button("Ask"):
        st.info("Your Question: " + question)

        st.info("Your Answer")
        answer, metadata = process_answer(question)
        st.write(answer)
        st.write(metadata)


if __name__ == '__main__':
    main()


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-65b3a98f-2c5fa1f236958f3059949b58;598e2151-facd-40b1-bf73-265378bc704c)

Repository Not Found for url: https://huggingface.co/LaMini-T5-738M/resolve/main/pytorch_model.bin.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.